In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import requests

# Fetch the JSON data from the URL
url = 'https://svs.gsfc.nasa.gov/vis/a000000/a005000/a005073/cities-eclipse-2024.json'
response = requests.get(url)
data = response.json()

# Load the data into a pandas DataFrame
df = pd.json_normalize(data)

# Data preprocessing
df['ECLIPSE_START'] = pd.to_datetime(df['ECLIPSE'].apply(lambda x: x[0]), format='%H:%M:%S')
df['ECLIPSE_START'] = df['ECLIPSE_START'].dt.hour * 60 + df['ECLIPSE_START'].dt.minute
X = df[['LAT', 'LON']]
y = df['ECLIPSE_START']

# Scaling features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)

# Define a DNN model
model = keras.Sequential([
    layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

# Checking for GPU and using it
if len(tf.config.list_physical_devices('GPU')) > 0:
    print("Using GPU for training")
else:
    print("Using CPU for training")

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=10, validation_split=0.2)

# Evaluate the model
mse = model.evaluate(X_test, y_test)
print(f'Mean Squared Error: {mse}')


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Assuming you have a trained model and test set ready
# X_test is your testing feature set, and y_test is the actual target values for the testing set

# Generate predictions
predictions = model.predict(X_test)

# Now you can calculate the performance metrics
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R²): {r2}')


In [ ]:
import numpy as np
from tensorflow import keras

# Assuming the model is already trained and available in the session
# If you need to load it, you can use: model = keras.models.load_model('path_to_your_saved_model')

def predict_eclipse_time(model, lat, lon, scaler):
    # Convert user input into the format the model expects, here assuming it needs a 2D array
    input_features = np.array([[lat, lon]])

    # Assuming the scaler used in the training phase is also available
    # If you need to load it, ensure you have it saved and accessible
    input_features_scaled = scaler.transform(input_features)

    # Making prediction
    predicted_time = model.predict(input_features_scaled)

    # Convert the predicted time from model output (if necessary, e.g., to HH:MM format)
    return predicted_time

# Main function to handle user interaction
def main(model, scaler):
    print("Please enter your coordinates to predict the eclipse time.")

    # User inputs latitude and longitude
    lat = float(input("Enter latitude (e.g., 34.0522): "))
    lon = float(input("Enter longitude (e.g., -118.2437): "))

    # Make prediction
    prediction = predict_eclipse_time(model, lat, lon, scaler)
    print(f"Predicted eclipse start time (in model's output format, e.g., minutes past midnight): {prediction[0]}")

    # Constants
    minutes_from_start = prediction[0]

    # Calculate hours and minutes
    total_hours = minutes_from_start / 60
    hours = int(total_hours) - 6
    minutes = (total_hours - hours) * 60

    print(hours-6, minutes)

# Assuming 'model' and 'scaler' are defined in your session
main(model, scaler)


